# Setup

In [1]:
import torch
import torch.nn as nn

import pickle

# Model designs

## CNN feature generator

In [4]:
class CNNFeatureGenerator(nn.Module):
    def __init__(self, num_features):
        super(CNNFeatureGenerator, self).__init__()
        
        self.feature_extractor = nn.Sequential(
            nn.Conv1d(in_channels=num_features, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
    
    def forward(self, all_features):
        """
        Forward pass through the feature extractor.

        Args:
            all_features: Tensor of shape (batch_size, num_features, window_size)

        Returns:
            Tensor: Extracted features of shape (batch_size, 64)
        """
        features = self.feature_extractor(all_features)
        features = features.view(features.size(0), -1)
        return features

## FCN Regressor

In [ ]:
class FCNRegressor(nn.Module):
    def __init__(self, feature_size=64):
        super(FCNRegressor, self).__init__()
        
        self.regressor = nn.Sequential(
            nn.Linear(feature_size, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    
    def forward(self, cnn_features):
        """
        Forward pass through the FCN regressor.

        Args:
            cnn_features: Tensor of shape (batch_size, feature_size)

        Returns:
            Tensor: Predicted energy expenditure (batch_size, 1)
        """
        output = self.regressor(cnn_features)
        return output

## CNN to FCN Model

In [8]:
class CNNFCNRegressor(nn.Module):
    def __init__(self, num_features=8):
        super(CNNFCNRegressor, self).__init__()
        
        self.cnn_feature_generator = CNNFeatureGenerator(num_features=num_features)
        self.fcn_regressor = FCNRegressor()
    
    def forward(self, all_features):
        """
        Forward pass through the combined model.

        Args:
            all_features: Tensor of shape (batch_size, num_features, window_size)

        Returns:
            Tensor: Predicted energy expenditure (batch_size, 1)
        """
        cnn_features = self.cnn_feature_generator(all_features)  # Extract features with CNN
        output = self.fcn_regressor(cnn_features)  # Regress with FCN
        return output

## LSTM Module

In [9]:
class LSTMModule(nn.Module):
    def __init__(self, input_size=64, hidden_size=64, num_layers=2):
        super(LSTMModule, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.5
        )
    
    def forward(self, cnn_features):
        """
        Forward pass through the LSTM module.

        Args:
            cnn_features: Tensor of shape (batch_size, seq_len, input_size)

        Returns:
            Tensor: Output features from the LSTM (batch_size, hidden_size)
        """
        lstm_out, _ = self.lstm(cnn_features)  # lstm_out: (batch_size, seq_len, hidden_size)
        lstm_features = lstm_out[:, -1, :]  # Take the last timestep (batch_size, hidden_size)
        return lstm_features

## CNN to LSTM to FCN Model

In [ ]:
class CNNLSTMFCNRegressor(nn.Module):
    def __init__(self, cnn_input_features=8, lstm_input_size=64, hidden_size=64, num_layers=2):
        super(CNNLSTMFCNRegressor, self).__init__()
        
        self.cnn = CNNFeatureGenerator(num_features=cnn_input_features)
        self.lstm = LSTMModule(input_size=lstm_input_size, hidden_size=hidden_size, num_layers=num_layers)
        self.fcn = FCNRegressor(feature_size=hidden_size)  # Feature size matches LSTM hidden size
    
    def forward(self, all_features):
        """
        Forward pass through the CNN-LSTM-FCN regressor.

        Args:
            all_features: Tensor of shape (batch_size, num_features, window_size)

        Returns:
            Tensor: Predicted energy expenditure (batch_size, 1)
        """
        cnn_features = self.cnn(all_features)  # Extract features from CNN (batch_size, feature_size)
        lstm_input = cnn_features.unsqueeze(1)  # Add temporal dimension (batch_size, seq_len=1, feature_size)
        lstm_features = self.lstm(lstm_input)  # Pass through LSTM (batch_size, hidden_size)
        output = self.fcn(lstm_features)  # Pass through FCN regressor (batch_size, 1)
        return output